# Análisis Exploratorio de Datos y Depuración
# Recaudo de Rentas Cedidas - Series de Tiempo 2026

---

## 📊 Objetivo

Este cuaderno implementa el **diagnóstico completo y depuración estructural** de los datos de recaudo para preparar el modelado predictivo con técnicas de series de tiempo de última generación (2026).

### Alcance del Análisis:
1. **Exploración inicial**: Estructura, dimensiones, tipos de datos
2. **Diagnóstico de calidad**: Valores faltantes, duplicados, inconsistencias
3. **Identificación de anomalías**: Outliers extremos, valores negativos, transacciones en escala cero
4. **Depuración estructural**: Limpieza, normalización, validación de integridad
5. **Análisis temporal**: Rangos, gaps, estacionalidad preliminar

---

**Autor**: Sistema de Análisis Predictivo  
**Fecha**: Febrero 2026  
**Versión**: 1.0

## 1. Configuración del Entorno

In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
import missingno as msno
from scipy import stats
from sklearn.ensemble import IsolationForest

# Configuración de visualización
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Bibliotecas cargadas exitosamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Carga de Datos

In [ ]:
# Cargar datos desde Excel
print("📁 Cargando datos de BaseRentasCedidas...")
df_raw = pd.read_excel('../BaseRentasCedidas (1).xlsx')

print(f"\n✅ Datos cargados exitosamente")
print(f"📊 Dimensiones: {df_raw.shape[0]:,} filas × {df_raw.shape[1]} columnas")
print(f"💾 Memoria utilizada: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 3. Exploración Inicial de Estructura

In [ ]:
# Información general del dataset
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("=" * 80)
df_raw.info()

print("\n" + "=" * 80)
print("PRIMERAS 10 FILAS")
print("=" * 80)
display(df_raw.head(10))

print("\n" + "=" * 80)
print("COLUMNAS DISPONIBLES")
print("=" * 80)
for i, col in enumerate(df_raw.columns, 1):
    print(f"{i:2d}. {col}")

## 4. Identificación de Columnas Clave para Series de Tiempo

Para el análisis de series de tiempo, necesitamos identificar:
- **Variable temporal**: Vigencia, Fecha, Periodo, etc.
- **Variable objetivo (target)**: Valor de recaudo
- **Variables auxiliares**: Tipo de renta, entidad, categoría, etc.

In [ ]:
# Identificar columnas con datos temporales y de recaudo
columnas_temporales = [col for col in df_raw.columns if any(x in col.lower() for x in ['vigencia', 'fecha', 'periodo', 'año', 'mes'])]
columnas_recaudo = [col for col in df_raw.columns if any(x in col.lower() for x in ['recaudo', 'valor', 'monto', 'ingreso'])]

print("🗓️  COLUMNAS TEMPORALES DETECTADAS:")
for col in columnas_temporales:
    print(f"  - {col}")
    if col in df_raw.columns:
        print(f"    Valores únicos: {df_raw[col].nunique()}")
        print(f"    Rango: {df_raw[col].min()} - {df_raw[col].max()}")
        print(f"    Tipo: {df_raw[col].dtype}\n")

print("\n💰 COLUMNAS DE RECAUDO DETECTADAS:")
for col in columnas_recaudo:
    print(f"  - {col}")
    if col in df_raw.columns:
        print(f"    Valores no nulos: {df_raw[col].notna().sum():,} ({df_raw[col].notna().sum()/len(df_raw)*100:.1f}%)")
        print(f"    Tipo: {df_raw[col].dtype}\n")

## 5. Diagnóstico de Calidad de Datos

In [ ]:
# Análisis de valores faltantes
print("=" * 80)
print("ANÁLISIS DE VALORES FALTANTES")
print("=" * 80)

missing_data = pd.DataFrame({
    'Columna': df_raw.columns,
    'Valores_Faltantes': df_raw.isnull().sum(),
    'Porcentaje': (df_raw.isnull().sum() / len(df_raw) * 100).round(2)
}).sort_values('Porcentaje', ascending=False)

missing_data = missing_data[missing_data['Valores_Faltantes'] > 0]

if len(missing_data) > 0:
    display(missing_data)
    
    # Visualización con missingno
    fig, ax = plt.subplots(figsize=(14, 6))
    msno.matrix(df_raw, ax=ax, sparkline=False)
    plt.title('Matriz de Valores Faltantes', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("✅ No se detectaron valores faltantes")

In [ ]:
# Análisis de duplicados
print("=" * 80)
print("ANÁLISIS DE REGISTROS DUPLICADOS")
print("=" * 80)

duplicados_totales = df_raw.duplicated().sum()
print(f"Duplicados completos: {duplicados_totales:,} ({duplicados_totales/len(df_raw)*100:.2f}%)")

if duplicados_totales > 0:
    print("\n⚠️  Se detectaron registros duplicados que requieren revisión")
else:
    print("✅ No se detectaron duplicados completos")

## 6. Preparación de Datos para Análisis Temporal

**NOTA IMPORTANTE**: Esta sección debe ajustarse según las columnas reales del dataset.  
Asumimos que existe una columna de `Vigencia` (año) y una columna de valor de recaudo.

In [ ]:
# TODO: AJUSTAR NOMBRES DE COLUMNAS SEGÚN DATASET REAL

# Verificar si existen columnas clave
if columnas_recaudo:
    columna_recaudo = columnas_recaudo[0]  # Tomar la primera columna detectada
    print(f"✅ Usando columna de recaudo: '{columna_recaudo}'")
else:
    print("⚠️  No se detectó automáticamente la columna de recaudo")
    print("Por favor, especificar manualmente en la siguiente celda")
    columna_recaudo = None

if columnas_temporales:
    columna_temporal = columnas_temporales[0]  # Tomar la primera columna detectada
    print(f"✅ Usando columna temporal: '{columna_temporal}'")
else:
    print("⚠️  No se detectó automáticamente la columna temporal")
    print("Por favor, especificar manualmente en la siguiente celda")
    columna_temporal = None

## 7. Análisis Estadístico de la Variable Objetivo (Recaudo)

In [ ]:
if columna_recaudo and columna_recaudo in df_raw.columns:
    print("=" * 80)
    print(f"ESTADÍSTICAS DESCRIPTIVAS: {columna_recaudo}")
    print("=" * 80)
    
    # Filtrar valores numéricos válidos
    recaudo_valido = df_raw[columna_recaudo].replace([np.inf, -np.inf], np.nan).dropna()
    
    if len(recaudo_valido) > 0:
        estadisticas = recaudo_valido.describe()
        print(estadisticas)
        
        # Estadísticas adicionales
        print(f"\n📊 ESTADÍSTICAS ADICIONALES:")
        print(f"  Mediana: ${recaudo_valido.median():,.2f}")
        print(f"  Moda: ${recaudo_valido.mode()[0]:,.2f}")
        print(f"  Coef. Variación: {(recaudo_valido.std()/recaudo_valido.mean()*100):.2f}%")
        print(f"  Asimetría (Skewness): {recaudo_valido.skew():.2f}")
        print(f"  Curtosis: {recaudo_valido.kurtosis():.2f}")
        
        # Identificar valores negativos
        negativos = (recaudo_valido < 0).sum()
        print(f"\n⚠️  Valores negativos: {negativos:,} ({negativos/len(recaudo_valido)*100:.2f}%)")
        
        # Identificar valores cercanos a cero
        cerca_cero = ((recaudo_valido >= 0) & (recaudo_valido < 1000)).sum()
        print(f"📉 Valores < $1,000: {cerca_cero:,} ({cerca_cero/len(recaudo_valido)*100:.2f}%)")
        
        # Identificar outliers extremos (>800M según plan)
        outliers_extremos = (recaudo_valido > 8e8).sum()
        print(f"🚨 Valores > $800M: {outliers_extremos:,} ({outliers_extremos/len(recaudo_valido)*100:.2f}%)")
    else:
        print("⚠️  No hay valores válidos para analizar")
else:
    print("⚠️  Columna de recaudo no identificada")

## 8. Visualización de Distribución de Recaudo

In [ ]:
if columna_recaudo and columna_recaudo in df_raw.columns:
    recaudo_valido = df_raw[columna_recaudo].replace([np.inf, -np.inf], np.nan).dropna()
    
    if len(recaudo_valido) > 0:
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Distribución de Recaudo', 'Distribución Log-Normal',
                          'Box Plot', 'Densidad de Probabilidad'),
            specs=[[{}, {}], [{}, {}]]
        )
        
        # Histograma normal
        fig.add_trace(
            go.Histogram(x=recaudo_valido, name='Recaudo', nbinsx=50),
            row=1, col=1
        )
        
        # Histograma logarítmico
        recaudo_positivo = recaudo_valido[recaudo_valido > 0]
        fig.add_trace(
            go.Histogram(x=np.log10(recaudo_positivo), name='Log10(Recaudo)', nbinsx=50),
            row=1, col=2
        )
        
        # Box plot
        fig.add_trace(
            go.Box(y=recaudo_valido, name='Recaudo'),
            row=2, col=1
        )
        
        # Densidad
        from scipy.stats import gaussian_kde
        if len(recaudo_positivo) > 1:
            kde = gaussian_kde(recaudo_positivo.sample(min(10000, len(recaudo_positivo))))
            x_range = np.linspace(recaudo_positivo.min(), recaudo_positivo.quantile(0.99), 100)
            fig.add_trace(
                go.Scatter(x=x_range, y=kde(x_range), mode='lines', name='Densidad'),
                row=2, col=2
            )
        
        fig.update_layout(height=800, showlegend=False, title_text="Análisis de Distribución del Recaudo")
        fig.show()

## 9. Detección de Outliers con Isolation Forest

In [ ]:
if columna_recaudo and columna_recaudo in df_raw.columns:
    recaudo_valido = df_raw[columna_recaudo].replace([np.inf, -np.inf], np.nan).dropna()
    
    if len(recaudo_valido) > 100:
        print("🔍 Detectando outliers con Isolation Forest...")
        
        # Preparar datos
        X = recaudo_valido.values.reshape(-1, 1)
        
        # Entrenar modelo
        iso_forest = IsolationForest(contamination=0.05, random_state=42)
        outlier_labels = iso_forest.fit_predict(X)
        
        # Contar outliers
        n_outliers = (outlier_labels == -1).sum()
        print(f"\n⚠️  Outliers detectados: {n_outliers:,} ({n_outliers/len(recaudo_valido)*100:.2f}%)")
        
        # Valores extremos
        outliers = recaudo_valido[outlier_labels == -1]
        print(f"\nTop 10 outliers más extremos:")
        print(outliers.nlargest(10).apply(lambda x: f"${x:,.2f}"))

## 10. Análisis Temporal Preliminar

Explorar cómo se distribuye el recaudo a lo largo del tiempo

In [ ]:
# Este código necesita ajustarse según la estructura real de los datos
# Por ahora, dejamos placeholder para completar una vez se explore el dataset

print("⏳ Este análisis se completará una vez se identifique la estructura temporal de los datos")
print("\nPróximos pasos:")
print("1. Identificar columnas de fecha/periodo")
print("2. Agregar recaudo por periodo (mensual/trimestral/semestral)")
print("3. Analizar estacionalidad y tendencias")
print("4. Identificar gaps temporales")

## 11. Resumen Ejecutivo de Diagnóstico

### Hallazgos Críticos Identificados

In [ ]:
print("=" * 80)
print("RESUMEN EJECUTIVO - DIAGNÓSTICO DE DATOS")
print("=" * 80)

print("\n📊 DIMENSIONES:")
print(f"  Registros totales: {len(df_raw):,}")
print(f"  Columnas: {df_raw.shape[1]}")

if columna_recaudo and columna_recaudo in df_raw.columns:
    recaudo_valido = df_raw[columna_recaudo].replace([np.inf, -np.inf], np.nan).dropna()
    
    print("\n⚠️  PROBLEMAS DETECTADOS:")
    
    # Valores negativos
    negativos = (recaudo_valido < 0).sum()
    if negativos > 0:
        print(f"  ❌ Valores negativos: {negativos:,} registros")
    
    # Valores extremos
    extremos = (recaudo_valido > 8e8).sum()
    if extremos > 0:
        print(f"  🚨 Outliers extremos (>$800M): {extremos:,} registros")
    
    # Valores cercanos a cero
    cerca_cero = ((recaudo_valido >= 0) & (recaudo_valido < 1000)).sum()
    if cerca_cero > 10000:
        print(f"  📉 Transacciones en escala cero (<$1,000): {cerca_cero:,} registros")
    
    print("\n✅ ACCIONES REQUERIDAS:")
    print("  1. Auditar y corregir valores negativos")
    print("  2. Aplicar Winsorization/Capping a outliers extremos")
    print("  3. Filtrar o normalizar transacciones de bajo valor")
    print("  4. Validar integridad temporal")
    print("  5. Preparar agregaciones por horizonte temporal")

print("\n" + "=" * 80)

## 12. Exportar Datos para Siguiente Fase

Guardar datos crudos para continuar con Feature Engineering

In [ ]:
# Guardar copia de datos originales
output_path = '../data/raw/datos_originales.parquet'
df_raw.to_parquet(output_path, index=False)
print(f"✅ Datos originales guardados en: {output_path}")

print("\n📋 SIGUIENTE PASO:")
print("   Abrir cuaderno: 02_Feature_Engineering.ipynb")